In [1]:
#installing libraries
!pip install selenium
!pip install scrapy
!pip install crochet
!pip install telegram
!pip install scrapy-rotating-proxies
!pip install scrapy_proxy_pool
!pip install shadow-useragent
!pip install scrapy-useragents
!pip install rotating-free-proxies
!pip install scrapy-user-agents
!pip install scrapy-random-useragent-pro
!pip install fake_headers
!pip install aiohttp
!pip install webdriver_manager



In [2]:

# Importing Libraries
import csv
import os
import time
import ujson
from random import randint
from typing import Dict, List, Any
import requests
from bs4 import BeautifulSoup

from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

from selenium import webdriver

from selenium.webdriver.chrome.options import Options



In [3]:
# Delete files if present
try:
    os.remove('Authors_URL.txt')
    os.remove('scraper_results.json')
except OSError:
    pass


In [4]:
#Writing Authors to a File:
def write_authors(list1, file_name):
    with open(file_name, 'w', encoding='utf-8') as f:
        for item in list1:
            f.write(item + '\n')

In [5]:

def initCrawlerScraper(seed):
    # Initialize driver for Chrome
    webOpt = webdriver.ChromeOptions()
    webOpt.add_experimental_option('excludeSwitches', ['enable-logging'])
    webOpt.add_argument('--ignore-certificate-errors')
    webOpt.add_argument('--incognito')
    webOpt.add_argument('--headless')  # Updated to use headless mode
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=webOpt)
    driver.get(seed)  # Start with the original link

    Links = []  # Array with pureportal profiles URL
    pub_data = []  # To store publication information for each pureportal profile

    nextLink = True
    print("Crawler has begun...")
    while nextLink:
        page = driver.page_source
        # XML parser to parse each URL
        bs = BeautifulSoup(page, "lxml")

        # Extracting exact URL by splitting the string into a list
        for link in bs.findAll('a', class_='link person'):
            url = str(link)[str(link).find('https://pureportal.coventry.ac.uk/en/persons/'):].split('"')
            Links.append(url[0])
        # Click on Next button to visit the next page
        try:
            if driver.find_elements(By.CSS_SELECTOR, ".nextLink"):
                element = driver.find_element(By.CSS_SELECTOR, ".nextLink")
                driver.execute_script("arguments[0].click();", element)
            else:
                nextLink = False
        except NoSuchElementException:
            break
    print("Crawler has found", len(Links), "pureportal profiles")
    write_authors(Links, 'Authors_URL.txt')

    print("Scraping publication data for", len(Links), "pureportal profiles...")
    count = 0
    for link in Links:
        # Visit each link to get data
        time.sleep(1)
        driver.get(link)
        try:
            if driver.find_elements(By.CSS_SELECTOR, ".portal_link.btn-primary.btn-large"):
                element = driver.find_elements(By.CSS_SELECTOR, ".portal_link.btn-primary.btn-large")
                for a in element:
                    if "research output".lower() in a.text.lower():
                        driver.execute_script("arguments[0].click();", a)
                        driver.get(driver.current_url)
                        # Get name of Author
                        name = driver.find_element(By.CSS_SELECTOR, "div.header.person-details > h1")
                        r = requests.get(driver.current_url)
                        # Parse all the data via BeautifulSoup
                        soup = BeautifulSoup(r.content, 'lxml')

                        # Extract publication name, publication URL, date, and CU Authors
                        table = soup.find('ul', attrs={'class': 'list-results'})
                        if table is not None:
                            for row in table.findAll('div', attrs={'class': 'result-container'}):
                                data = {}
                                data['name'] = row.h3.a.text
                                data['pub_url'] = row.h3.a['href']
                                date = row.find("span", class_="date")

                                rowitem = row.find_all(['div'])
                                span = row.find_all(['span'])
                                data['cu_author'] = name.text
                                data['date'] = date.text
                                print("Publication Name:", row.h3.a.text)
                                print("Publication URL:", row.h3.a['href'])
                                print("CU Author:", name.text)
                                print("Date:", date.text)
                                print("\n")
                                pub_data.append(data)
            else:
                # Get name of Author
                name = driver.find_element(By.CSS_SELECTOR, "div.header.person-details > h1")
                r = requests.get(link)
                # Parse all the data via BeautifulSoup
                soup = BeautifulSoup(r.content, 'lxml')
                # Extract publication name, publication URL, date, and CU Authors
                table = soup.find('div', attrs={'class': 'relation-list relation-list-publications'})
                if table is not None:
                    for row in table.findAll('div', attrs={'class': 'result-container'}):
                        data = {}
                        data["name"] = row.h3.a.text
                        data['pub_url'] = row.h3.a['href']
                        date = row.find("span", class_="date")
                        rowitem = row.find_all(['div'])
                        span = row.find_all(['span'])
                        data['cu_author'] = name.text
                        data['date'] = date.text
                        print("Publication Name:", row.h3.a.text)
                        print("Publication URL:", row.h3.a['href'])
                        print("CU Author:", name.text)
                        print("Date:", date.text)
                        print("\n")
                        pub_data.append(data)
        except Exception:
            continue

    print("Crawler has scraped data for", len(pub_data), "pureportal publications")
    driver.quit()
    # Writing all the scraped results to a file in JSON format
    with open('scraper_results.json', 'w') as f:
        ujson.dump(pub_data, f)


In [ ]:
#Initializing Crawler and Scrape
initCrawlerScraper('https://pureportal.coventry.ac.uk/en/publications/')

C:\Users\Prayas\AppData\Local\Temp\ipykernel_8336\2360245199.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=webOpt)


Crawler has begun...
Crawler has found 24900 pureportal profiles
Scraping publication data for 24900 pureportal profiles...
Publication Name: Age-Related Differences in Trunk Kinematics and Interplanar Decoupling with the Pelvis during Gait in Healthy Older versus Younger Men
Publication URL: https://pureportal.coventry.ac.uk/en/publications/age-related-differences-in-trunk-kinematics-and-interplanar-decou
CU Author: Michael Duncan
Date: 18 Apr 2023


Publication Name: Artificial Intelligence for skeleton-based physical rehabilitation action evaluation: A systematic review
Publication URL: https://pureportal.coventry.ac.uk/en/publications/artificial-intelligence-for-skeleton-based-physical-rehabilitatio
CU Author: Michael Duncan
Date: May 2023


Publication Name: Association of minimally processed and ultra-processed food daily consumption with obesity in overweight adults:  a cross-sectional study
Publication URL: https://pureportal.coventry.ac.uk/en/publications/association-of-minima

Publication Name: Perception of Affordances for Dribbling in Soccer: Exploring Children as Architects of Skill Development Opportunity
Publication URL: https://pureportal.coventry.ac.uk/en/publications/perception-of-affordances-for-dribbling-in-soccer-exploring-child
CU Author: Michael Duncan
Date: 24 Jun 2022


Publication Name: Perceptions and practices of fundamental movement skills in grassroots soccer coaches
Publication URL: https://pureportal.coventry.ac.uk/en/publications/perceptions-and-practices-of-fundamental-movement-skills-in-grass
CU Author: Michael Duncan
Date: Aug 2022


Publication Name: Physical activity during school-time and fundamental movement skills: a study among pre-schoolers with and without Physical Education classes
Publication URL: https://pureportal.coventry.ac.uk/en/publications/physical-activity-during-school-time-and-fundamental-movement-ski
CU Author: Michael Duncan
Date: 13 Jun 2022


Publication Name: Practices of Strength and Conditioning Coaches: A

Publication Name: A comparison of placebo and nocebo effects on objective and subjective postural stability: a double-edged sword?
Publication URL: https://pureportal.coventry.ac.uk/en/publications/a-comparison-of-placebo-and-nocebo-effects-on-objective-and-subje
CU Author: Michael Duncan
Date: 18 Aug 2022


Publication Name: Actual and perceived motor competence mediate the relationship between physical fitness and technical skill performance in young soccer players
Publication URL: https://pureportal.coventry.ac.uk/en/publications/actual-and-perceived-motor-competence-mediate-the-relationship-be
CU Author: Michael Duncan
Date: 3 Aug 2022


Publication Name: Acute effects of maximal versus submaximal hurdle jump exercises on measures of balance, reactive strength, vertical jump performance and leg stiffness in youth volleyball players
Publication URL: https://pureportal.coventry.ac.uk/en/publications/acute-effects-of-maximal-versus-submaximal-hurdle-jump-exercises-
CU Author: Michael 

Publication Name: The Acute Effects of Cognitively Demanding Physical Activity on Inhibitory and Affective Responses in Children: An Online-Based Mixed Methods Approach
Publication URL: https://pureportal.coventry.ac.uk/en/publications/the-acute-effects-of-cognitively-demanding-physical-activity-on-i
CU Author: Michael Duncan
Date: 2 Dec 2022


Publication Name: The Effect of 12-Weeks Recreational Football (Soccer) for Health Intervention on Functional Movement in Older Adults
Publication URL: https://pureportal.coventry.ac.uk/en/publications/the-effect-of-12-weeks-recreational-football-soccer-for-health-in
CU Author: Michael Duncan
Date: 20 Oct 2022


Publication Name: Co-Designing and Refining a Home-Based Exercise Programme for Adults Living with Overweight and Obesity: Insight from People with Lived Experience
Publication URL: https://pureportal.coventry.ac.uk/en/publications/co-designing-and-refining-a-home-based-exercise-programme-for-adu
CU Author: David Broom
Date: 1 Apr 2023



...

Publication Name: Movement Velocity during High- and Low-Velocity Resistance Exercise Protocols in Older Adults
Publication URL: https://pureportal.coventry.ac.uk/en/publications/movement-velocity-during-high-and-low-velocity-resistance-exercis
CU Author: Neil Clarke
Date: Jul 2018


Publication Name: Next Day Subjective and Objective Recovery Indices Following Acute Low and High Training Loads in Academy Rugby Union Players
Publication URL: https://pureportal.coventry.ac.uk/en/publications/next-day-subjective-and-objective-recovery-indices-following-acut
CU Author: Neil Clarke
Date: 15 Jun 2018


Publication Name: Reliability, validity and measurement error
Publication URL: https://pureportal.coventry.ac.uk/en/publications/reliability-validity-and-measurement-error
CU Author: Neil Clarke
Date: Oct 2018


Publication Name: Seasonal changes in physical qualities of elite youth soccer players according to maturity status: comparisons with aged matched controls
Publication URL: https://pu

Publication Name: Civil Society Organizations work to end Cameroon’s Anglophone Conflict
Publication URL: https://pureportal.coventry.ac.uk/en/publications/civil-society-organizations-work-to-end-cameroons-anglophone-conf
CU Author: Nancy Annan
Date: 6 Feb 2023


Publication Name: From ‘Anglophone Problem’ to ‘Anglophone Conflict’ in Cameroon: Assessing Prospects for Peace
Publication URL: https://pureportal.coventry.ac.uk/en/publications/from-anglophone-problem-to-anglophone-conflict-in-cameroon-assess
CU Author: Nancy Annan
Date: Apr 2023


Publication Name: War and no peace in Southern Cameroons
Publication URL: https://pureportal.coventry.ac.uk/en/publications/war-and-no-peace-in-southern-cameroons
CU Author: Nancy Annan
Date: 16 Jan 2023


Publication Name: Voices from 'Ground Zero': Interrogating History, Culture and Identity in the Resolution of Cameroon's Anglophone Conflict
Publication URL: https://pureportal.coventry.ac.uk/en/publications/voices-from-ground-zero-interrogating

Publication Name: Degree correlations in directed scale-free networks
Publication URL: https://pureportal.coventry.ac.uk/en/publications/degree-correlations-in-directed-scale-free-networks
CU Author: Charo del Genio
Date: 2014


Publication Name: The structure and dynamics of multilayer networks
Publication URL: https://pureportal.coventry.ac.uk/en/publications/the-structure-and-dynamics-of-multilayer-networks
CU Author: Charo del Genio
Date: 1 Nov 2014


Publication Name: Endemic infections are always possible on regular networks
Publication URL: https://pureportal.coventry.ac.uk/en/publications/endemic-infections-are-always-possible-on-regular-networks
CU Author: Charo del Genio
Date: 25 Oct 2013


Publication Name: Constructing and sampling directed graphs with given degree sequences
Publication URL: https://pureportal.coventry.ac.uk/en/publications/constructing-and-sampling-directed-graphs-with-given-degree-seque
CU Author: Charo del Genio
Date: 6 Feb 2012


Publication Name: All s

Publication Name: Capability, Opportunity, and Motivation—Identifying Constructs for Increasing Physical Activity Behaviours in Women with Polycystic Ovary Syndrome (PCOS)
Publication URL: https://pureportal.coventry.ac.uk/en/publications/capability-opportunity-and-motivationidentifying-constructs-for-i
CU Author: Cain Clark
Date: 28 Jan 2023


Publication Name: Circulating levels of adropin and diabetes: a systematic review and meta-analysis of observational studies
Publication URL: https://pureportal.coventry.ac.uk/en/publications/circulating-levels-of-adropin-and-diabetes-a-systematic-review-an
CU Author: Cain Clark
Date: 7 Apr 2023


Publication Name: Dietary Intake and Body Composition During Ramadan in Athletes: A Systematic Review and Meta-Analysis With Meta-Regression
Publication URL: https://pureportal.coventry.ac.uk/en/publications/dietary-intake-and-body-composition-during-ramadan-in-athletes-a-
CU Author: Cain Clark
Date: 2023


Publication Name: Effect of magnesium supplem

Publication Name: The Good Behaviour Game: Maintaining students’ physical distancing in physical education classes during the COVID-19 pandemic
Publication URL: https://pureportal.coventry.ac.uk/en/publications/the-good-behaviour-game-maintaining-students-physical-distancing-
CU Author: Cain Clark
Date: Mar 2023


Publication Name: The impact of sport-discipline and sex on physical fitness and bone markers in athletes
Publication URL: https://pureportal.coventry.ac.uk/en/publications/the-impact-of-sport-discipline-and-sex-on-physical-fitness-and-bo
CU Author: Cain Clark
Date: 17 Feb 2023


Publication Name: The interaction between MC4R gene variant (rs17782313) and dominant dietary patterns on depression in obese and overweight women: a cross sectional study
Publication URL: https://pureportal.coventry.ac.uk/en/publications/the-interaction-between-mc4r-gene-variant-rs17782313-and-dominant
CU Author: Cain Clark
Date: Dec 2023


Publication Name: The problem-solving method: Efficacy for 